In [ ]:
if True:
    import numpy as np
    import panel as pn, holoviews as hv
    hv.extension('bokeh', logo=False)
    pn.extension('katex')

    # SciPy for robust generalized eigendecomposition and QZ if available, otherwise use numpy
    try:
        from scipy.linalg import eig as scipy_eig, eigh, cholesky, qz, solve
        HAVE_SCIPY = True
    except Exception:
        from numpy.linalg import eig as np_eig, eigh as np_eigh, cholesky as np_cholesky, solve as np_solve
        HAVE_SCIPY = False

    np.set_printoptions(precision=6, suppress=True)

    def lin_solve(A, B):
        if HAVE_SCIPY:
            return solve(A, B, assume_a='gen', overwrite_b=False)
        else:
            return np.linalg.solve(A, B)

    def chol_lower(B):
        if HAVE_SCIPY:
            L = cholesky(B, lower=True, overwrite_a=False, check_finite=True)
        else:
            U = np_cholesky(B)  # upper
            L = U.T
        return L

    def residual(A,B,lam,x):
        num = np.linalg.norm(A@x - lam*(B@x))
        den = (np.linalg.norm(A)*np.linalg.norm(x) + 1e-15)
        return num/den

#   print("HAVE_SCIPY =", HAVE_SCIPY)

    def _safe_symmetrize(M):
        return 0.5*(M + M.T)

    class GeneralizedRayleighQuotientViewer(pn.viewable.Viewer):
        """
        Visualizes the generalized Rayleigh quotient
            ρ(θ) = (x(θ)^T A^T A x(θ)) / (x(θ)^T B^T B x(θ)),
        with x(θ) = [cos θ, sin θ]^T for θ ∈ [0, π], and marks stationary θ
        given by the generalized eigenvectors of (A^T A, B^T B).

        Notes
        -----
        • Intended for 2×2 A, B.
        • Uses solves (not explicit inverses) for stability.
        • Eigenvectors are normalized and mapped to θ ∈ [0, π).
        """
        def __init__(self, A, B, ylim=None, **params):
            self.A = np.asarray(A, dtype=float)
            self.B = np.asarray(B, dtype=float)
            assert self.A.shape == (2,2) and self.B.shape == (2,2), "GSVDQuotientViewer expects 2×2 A and B."
            self.ylim=ylim

            super().__init__(**params)
            # Use symmetric forms to avoid numerical asymmetry
            AtA = _safe_symmetrize(self.A.T @ self.A)
            BtB = _safe_symmetrize(self.B.T @ self.B)

            # Sample θ ∈ [0, π] and evaluate ρ(θ)
            theta = np.linspace(0.0, np.pi, 500)
            rho_vals = []
            for t in theta:
                x = np.array([np.cos(t), np.sin(t)])
                num = float(x.T @ AtA @ x)
                den = float(x.T @ BtB @ x)
                rho_vals.append(num/den if den > 1e-14 else np.nan)

            self.theta = theta
            self.rho = np.array(rho_vals)

            # Generalized eigenproblem: (AtA) v = μ (BtB) v
            # Solve via Cholesky of BtB if SPD; else fall back to solve
            # Use np.linalg.eig on BtB^{-1} AtA in a stable way:
            try:
                # Solve BtB X = AtA (column-wise) without forming BtB^{-1}
                X = np.linalg.solve(BtB, AtA)
                eigvals, eigvecs = np.linalg.eig(X)
            except np.linalg.LinAlgError:
                # Add tiny Tikhonov for robustness (viewer only)
                eps = 1e-12
                X = np.linalg.solve(BtB + eps*np.eye(2), AtA)
                eigvals, eigvecs = np.linalg.eig(X)

            eigvals = np.real_if_close(eigvals)
            eigvecs = np.real_if_close(eigvecs)

            # Sort by eigenvalue ascending
            idx = np.argsort(eigvals)
            self.mu = np.array(eigvals[idx], dtype=float)
            V = np.array(eigvecs[:, idx], dtype=float)

            # Normalize eigenvectors to unit 2-norm and map to angles θ ∈ [0, π)
            xs = []
            for i in range(V.shape[1]):
                v = V[:, i]
                nv = np.linalg.norm(v)
                if nv == 0:
                    v = np.array([1.0, 0.0])
                else:
                    v = v / nv
                # Enforce a consistent sign (place in upper semicircle)
                if v[1] < 0 or (v[1] == 0 and v[0] < 0):
                    v = -v
                xs.append(v)
            self.xs = np.vstack(xs)

        def _get_plot(self):
            curve = hv.Curve((180./np.pi*self.theta, self.rho), 'θ', 'ρ(θ)').opts(width=560, height=320)
            if self.ylim:
                curve=curve.opts(ylim=self.ylim)
            pts = []
            vlines = []
            for x, mu in zip(self.xs, self.mu):
                t = 180/np.pi*(float(np.arctan2(x[1], x[0])) % np.pi)
                pts.append((t, float(mu)))
                vlines.append(hv.VLine(t).opts(color='red', line_width=1.5))
            markers = hv.Scatter(pts, 'θ', 'ρ').opts(color='red', size=8)
            plot = curve * markers
            for vline in vlines:
                plot = plot * vline
            return plot.opts(title='Generalized Rayleigh Quotient and Stationary Directions', show_grid=True)

        def _get_markdown(self):
            items = []
            for i, (x, mu) in enumerate(zip(self.xs, self.mu), start=1):
                t = 180/np.pi*float(np.arctan2(x[1], x[0]) % np.pi)
                text_md = f'**λ_{i}** = {mu:.2g},&ensp;&ensp; θ = {t:.2f} degrees'
                vec_latex = (
                    r'$$\qquad x_' + f'{i} = \\begin{{pmatrix}} ' +
                    f'{x[0]:.2f} \\\\ {x[1]:.2f}' +
                    r'\end{pmatrix}$$'
                )
                items.append(
                    pn.Column(
                        pn.pane.Markdown(text_md),
                        pn.pane.LaTeX(vec_latex)
                    )
                )
            return pn.Column(*items, width=320)

        def __panel__(self):
            return pn.Row(
                pn.pane.HoloViews(self._get_plot()),
                pn.Spacer(width=24),
                pn.Column(
                    pn.Spacer(height=8),
                    '### Generalized eigenvalues and eigenvectors',
                    pn.Row(pn.Spacer(width=8), self._get_markdown())
                )
            )

    def _chol_spd(M):
        try:
            return np.linalg.cholesky(M)
        except np.linalg.LinAlgError:
            eps = 1e-12
            return np.linalg.cholesky(M + eps*np.eye(M.shape[0]))

    def _b_orthonormalize(B, V):
        # B-orthonormalize columns of V
        Q = np.zeros_like(V, dtype=float)
        for i in range(V.shape[1]):
            v = V[:, i].astype(float)
            for j in range(i):
                v -= (Q[:, j].T @ B @ v) * Q[:, j]
            nB = np.sqrt(max(v.T @ B @ v, 0.0)) + 1e-15
            Q[:, i] = v / nB
        return Q

    def _generalized_eig(A, B):
        # Solve (B^{-1}A) v = λ v via linear solve; 2x2 teaching purpose
        X = np.linalg.solve(B, A)
        w, V = np.linalg.eig(X)
        w = np.real_if_close(w)
        V = np.real_if_close(V)
        Vb = _b_orthonormalize(B, V)
        return w, Vb

    class BGeometryViewer(pn.viewable.Viewer):
        """
        Left: original geometry in x-space: B-ellipsoid {x: x^T B x = 1} + generalized eigenvectors (B-orthonormal).
        Right: B-induced geometry in z-space via whitening (z = L^T x): unit circle + transformed eigenvectors (Euclidean-orthonormal).
        """
        def __init__(self, A, B, width=200, height=200, **params):
            super().__init__(**params)
            self.A = np.asarray(A, float)
            self.B = np.asarray(B, float)
            assert self.A.shape == (2,2) and self.B.shape == (2,2), "Use 2×2 A, B."

            self.width, self.height = int(width), int(height)

            # Cholesky whitening
            self.L = _chol_spd(self.B)

            # Ellipse in x-space: x = L^{-T} u, ||u||=1
            theta = np.linspace(0, 2*np.pi, 600)
            U = np.vstack([np.cos(theta), np.sin(theta)])   # unit circle
            X_ellipse = np.linalg.solve(self.L.T, U)        # x-points
            self.xy = (X_ellipse[0, :], X_ellipse[1, :])

            # z-space unit circle is just U (for display)
            self.zu = (U[0, :], U[1, :])

            # Generalized eigenvectors (B-orthonormal in x-space)
            self.w, self.Xeig = _generalized_eig(self.A, self.B)  # columns: x-eigenvectors (B-orthonormal)
            # Transform eigenvectors to z-space: z = L^T x; they should be Euclidean-orthonormal
            self.Zeig = self.L.T @ self.Xeig

            # Segment scale for vectors
            self.scale_x = 1.1 * np.max(np.linalg.norm(X_ellipse, axis=0))
            self.scale_z = 1.1  # unit circle radius

        def _segments_from_cols(self, M, scale):
            segs, pts = [], []
            for i in range(M.shape[1]):
                v = M[:, i].astype(float)
                v = v / (np.linalg.norm(v) + 1e-15)
                p = scale * v
                q = -p
                segs.append(((q[0], q[1]), (p[0], p[1])))
                pts.append((p[0], p[1]))
            return segs, pts

        def _plot_xspace(self):
            path = hv.Path([np.column_stack(self.xy)], kdims=['x','y']).opts(line_width=2)
            segs, pts = self._segments_from_cols(self.Xeig, self.scale_x)
            segs = hv.Segments(segs, kdims=['x0','y0','x1','y1']).opts(line_width=3)
            markers = hv.Points(pts, kdims=['x','y']).opts(size=6)

            title_txt = 'Original geometry (x-space)'
            return (path * segs * markers).opts(
                frame_width=self.width, aspect='square',
                xlabel='x₁', ylabel='x₂', show_grid=True, title=title_txt
            )

        def _plot_zspace(self):
            path = hv.Path([np.column_stack(self.zu)], kdims=['z1','z2']).opts(line_width=2)
            segs, pts = self._segments_from_cols(self.Zeig, self.scale_z)
            segs = hv.Segments(segs, kdims=['z10','z20','z11','z21']).opts(line_width=3)
            markers = hv.Points(pts, kdims=['z1','z2']).opts(size=6)

            title_txt = 'B-induced geometry (z-space)'
            return (path * segs * markers).opts(
                frame_width=self.width, aspect='square',
                xlabel='z₁', ylabel='z₂', show_grid=True, title=title_txt
            )

        def _info_panel(self):
            XtBX = self.Xeig.T @ self.B @ self.Xeig
            ZtZ  = self.Zeig.T @ self.Zeig
            lam_text = "Generalized eigenvalues: " + ", ".join(f"{wi:.3g}" for wi in self.w)
            gram_AB = (
                r"$\qquad A = \begin{pmatrix}"
                f"{self.A[0,0]:.1f} & {self.A[0,1]:.1f} \\\\ {self.A[1,0]:.1f} & {self.A[1,1]:.1f}"
                r"\end{pmatrix}$,"
                r"$\qquad B = \begin{pmatrix}"
                f"{self.B[0,0]:.1f} & {self.B[0,1]:.1f} \\\\ {self.B[1,0]:.1f} & {self.B[1,1]:.1f}"
                r"\end{pmatrix}$"
            )
            gram_X = (
                r"$\qquad X^T B X = \begin{pmatrix}"
                f"{XtBX[0,0]:.1f} & {XtBX[0,1]:.1f} \\\\ {XtBX[1,0]:.1f} & {XtBX[1,1]:.1f}"
                r"\end{pmatrix}$"
            )
            gram_Z = (
                r"$\qquad Z^T Z = \begin{pmatrix}"
                f"{ZtZ[0,0]:.1f} & {ZtZ[0,1]:.1f} \\\\ {ZtZ[1,0]:.1f} & {ZtZ[1,1]:.1f}"
                r"\end{pmatrix}$"
            )
            return pn.Column(
                pn.pane.Markdown("### GEP Results"),
                pn.pane.Markdown(lam_text),
                pn.pane.LaTeX(gram_AB),
                pn.pane.LaTeX(gram_X),
                pn.pane.LaTeX(gram_Z),
                width=280
            )

        def _caption_panel(self):
            return pn.Column(
                pn.Spacer(height=20),
                pn.pane.Markdown("### B-Orthogonality"),
                pn.pane.LaTeX(r"""<strong>Top Figure:</strong> the $B$-ellipsoid $\{x:\;x^T B x=1\}$ in the original $x$-space.<br>
    $\qquad$ The generalized eigenvectors are not orthogonal.<br><br>
    <strong>Bottom Figure:</strong> after whitening ($z=L^T x$ with $B=LL^T$),<br>
    $\qquad$ the constraint becomes the unit circle $\|z\|=1$ in $z$-space.<br>
    $\qquad$ The generalized eigenvectors are $B$-orthogonal"""))

        def __panel__(self):
            return pn.Column(pn.pane.Markdown("### B-Orthogonality"),
                             pn.Row( self._plot_xspace(),pn.Spacer(width=12),self._info_panel()),
                             pn.Row( self._plot_zspace(),pn.Spacer(width=12),self._caption_panel()))

<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">The Generalized Eigenvalue Problem</strong>
</div></div>

# 1. Generalized Eigenproblem and Matrix Pencils

## 1.1 Motivation

In many engineering problems, the eigenvalue equation does not appear in the simple form $A x = \lambda x$, but rather as

$\qquad A x = \lambda B x,$

with two square matrices $A,B \in \mathbb{R}^{n\times n}$.  

Examples:
- **Vibration analysis:** $Kx = \lambda Mx$, where $K$ is the stiffness matrix and $M$ the mass matrix.  
- **Control and stability:** Pencils $(A,B)$ arise in system stability when comparing dynamics to reference models.  
- **PDE discretizations:** Boundary-value problems often lead to generalized eigenproblems after discretization.

This motivates the study of the **generalized eigenvalue problem (GEP)** and the associated **matrix pencil**.

## 1.2 Definitions

<div style="background-color:#F2F5A9;color:black;">

**Definition:** Let $A,B \in \mathbb{R}^{n\times n}$.

- A scalar $\lambda \in \mathbb{C}$ is called a **generalized eigenvalue** of the pair $(A,B)$ if there exists a nonzero vector $x \in \mathbb{C}^n$ such that  
  $\qquad A x = \lambda B x.$  
  The vector $x$ is then a **right generalized eigenvector** associated with $\lambda$.

- A nonzero vector $y \in \mathbb{C}^n$ satisfying  
  $\qquad y^T A = \lambda y^T B$  
  is called a **left generalized eigenvector** corresponding to $\lambda$.

- The pair $(A,B)$ defines the **matrix pencil**  
  $\qquad \mathcal{L}(\lambda) = A - \lambda B.$  
</div>

<div style="background-color:#F2F5A9;color:black;">

**Definition:** Given a generalized eigenproblem $A x = \lambda B x\;\;$ with  $A,B \in \mathbb{R}^{n\times n}$
- The pencil $(A,B)$ is called **regular** if  
  $\qquad \det(A - \lambda B) \not\equiv 0,$  
  i.e. $\det(A - \lambda B)$ is a polynomial of degree $n$ in $\lambda$.  
  Otherwise the pencil is **singular**.

- The **finite generalized eigenvalues** of a regular pencil are precisely the roots of  
  $\qquad \det(A - \lambda B) = 0.$

- In the regular square case, the **eigenvalue at infinity** ($\lambda=\infty$) occurs iff $B$ is singular.  
  Equivalently, with $\mu = 1/\lambda$, zeros of
  $\;\; \det(B - \mu A)$ at $\mu=0$ represent eigenvalues at infinity.
</div>

## 1.3 Examples

### 1.3.1 Regular Pencil Examples

For regular (square) pencils in $\mathbb{R}^{n \times n}$ we find $n$ eigenvalues as before,<br>
$\qquad$ although some of the eigenvalues can be infinite.

##### **Example 1**

$\qquad 
A = \begin{pmatrix} 2 & 1 \\ 0 & 3 \end{pmatrix}, \quad
B = I = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}.
$

$\qquad 
\det(A - \lambda B) = \det\!\begin{pmatrix} 2-\lambda & 1 \\ 0 & 3-\lambda \end{pmatrix}
= (2-\lambda)(3-\lambda).
$

$\qquad$ This is a quadratic polynomial with eigenvalues $\lambda = 2,3$.

##### **Example 2**

$\qquad
A=\begin{pmatrix}2 & 1\\[2pt] 1 & 2\end{pmatrix},\quad
B=\begin{pmatrix}1 & 0\\[2pt] 0 & 2\end{pmatrix}.
$

$\qquad
\det(A-\lambda B)
=\det\!\begin{pmatrix}2-\lambda & 1\\[2pt] 1 & 2-2\lambda\end{pmatrix}
=2\lambda^2-6\lambda+3.
$

$\qquad
\lambda_{1,2}=\displaystyle{\frac{6\pm\sqrt{12}}{4}
=\frac{3\pm\sqrt{3}}{2}}.
$

##### **Example 3**

$\qquad
A = \begin{pmatrix} 1 & 0 \\ 0 & 2 \end{pmatrix}, \quad
B = \begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix}.
$

$\qquad
\det
\left( A - \lambda B \right) =
\det \begin{pmatrix}
1-\lambda & 0 \\
0 & 2
\end{pmatrix} = 2(1-\lambda).
$

$\qquad$ One finite eigenvalue: $\lambda=1$.

The missing root is interpreted as $\lambda=\infty$, because $B$ is singular (rank deficient by 1).<br><br>
$\qquad$ Indeed, setting $\mu = \frac{1}{\lambda}$  
$\qquad \det(B-\mu A)
=\det!\begin{pmatrix}1-\mu&0\\0&-2\mu\end{pmatrix}
=(1-\mu)(-2\mu)=-2\mu(1-\mu)\;\;$
shows an eigenvalue at $\mu = 0$.

$\qquad$ Mapping back: $\mu=1 \Rightarrow \lambda=1$; $\mu=0 \Rightarrow \lambda=\infty$.

So this pencil has the two eigenvalues ${1,\infty}$, consistent with $n=2$.

### 1.3.2 Singular Pencil Examples

For singular (square) pencils the naive "roots of $\det(A-\lambda B)$" picture breaks<br>
and will require further theory.

##### **Example 1**

$\qquad 
A = \begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix}, \quad
B = \begin{pmatrix} 0 & 0 \\ 0 & 0 \end{pmatrix}.
$

$\qquad 
A - \lambda B = A, \quad \det(A-\lambda B) = \det(A) = 0.
$

##### **Example 2**

$\qquad
A = \begin{pmatrix} 1 & 2 \\ 2 & 4 \end{pmatrix}, \quad
B = \begin{pmatrix} 0 & 1 \\ 0 & 2 \end{pmatrix}.
$

$\qquad
\det \left(A - \lambda B\right) =
\det \begin{pmatrix}
1 & 2 - \lambda \\
2 & 4 - 2\lambda
\end{pmatrix} = 0.
$

### 1.3.3 Application: Two-Mass Spring System Example

A simple physical model leads naturally to a generalized eigenproblem.  
Consider two equal masses $m=1$ connected by springs with stiffness $k_1=2$, $k_2=1$:

- The **mass matrix** is diagonal:  
$\qquad M = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}.$

- The **stiffness matrix** is  
$\qquad K = \left(\begin{array}{rr} k_1+k_2 & -k_2 \\ -k_2 & k_2 \end{array}\right)
= \left(\begin{array}{rr} 3 & -1 \\ -1 & 1 \end{array}\right).$

The vibration modes satisfy  
$\qquad Kx = \lambda Mx.$

- Eigenvalues $\lambda$ are **squared natural frequencies** $\omega^2$.  
- The corresponding eigenvectors are the **mode shapes** (relative displacements of the masses).

**Computation.** Solving $\det(K-\lambda M)=0$ gives  
$\qquad \lambda_{1,2} = \tfrac{4 \pm \sqrt{8}}{2} = 2 \pm \sqrt{2}.$  
Thus the natural frequencies are $\omega_{1,2}=\sqrt{\,2\pm\sqrt{2}\,}.$

**From eigenpairs to motion.** For each eigenpair $(\lambda_i, v_i)$ with $\omega_i=\sqrt{\lambda_i}$, the corresponding **pure-mode motion** is  
$\qquad x_{i}(t) = c_i\, v_i \cos(\omega_i t + \phi_i),$  
and the **general motion** is a superposition  
$\qquad x(t) = \sum_i c_i\, v_i \cos(\omega_i t + \phi_i),$  
where $c_i,\phi_i$ come from initial conditions. Mode shapes $v_i$ set the displacement ratios; $\omega_i$ set the oscillation speeds.

<details>
<summary><strong>Derivation: from eigenproblem to motion</strong></summary>

Starting from the equations of motion  
$\qquad M \ddot{x}(t) + K x(t) = 0,$  
seek harmonic solutions $x(t)=v e^{i\omega t}$. Substituting gives  
$\qquad (K - \omega^2 M) v = 0,$  
i.e., the **generalized eigenproblem** $K v = \lambda M v$ with $\lambda=\omega^2$.  
Hence each eigenvalue yields a frequency $\omega=\sqrt{\lambda}$ and each eigenvector $v$ yields a mode shape.  
The real motion uses $\cos(\omega t+\phi)$ with amplitudes/phases set by initial conditions.
</details>

In [ ]:
video_pane = pn.pane.Video("Figs/TwoMassSpringModes.mp4", loop=False, autoplay=False, width=640, height=300)

# Build the Panel layout
layout = pn.Column(
    "## Two Mass Spring System",
    "### The system has two vibration modes",
    video_pane,
)

# Serve the Panel app
layout.servable()

**Two-mass spring system.**  
Two equal masses are connected by springs and fixed to rigid walls.  
The generalized eigenvalue problem $Kx=\lambda Mx$ produces the vibration modes:  
- **Mode 1 (lower frequency):** both masses move in phase, stretching and compressing the system uniformly.  
- **Mode 2 (higher frequency):** the masses move out of phase, with one advancing while the other retreats.  
Each eigenvector gives a mode shape, and the eigenvalues determine the squared natural frequencies.

## 1.4 Variants of Matrix Pencils and the Generalized Eigenproblem

Before introducing the Rayleigh quotient, it is useful to situate the generalized eigenproblem in a few special cases.

- **Non-square Pencils.**  
  If $A \in \mathbb{R}^{m\times n}$ and $B \in \mathbb{R}^{m\times n}$ are rectangular, the determinant $\det(A-\lambda B)$ is not defined.  
  In this setting, the appropriate tool is the **generalized singular value decomposition (GSVD)**, which provides ratios of singular values rather than eigenvalues. We return to this later in the series.

- **Square Pencil Case 1: $B$ Invertible.**  
  If $B \in \mathbb{R}^{n\times n}$ is nonsingular, then  
  $\qquad A x = \lambda B x \iff B^{-1} A x = \lambda x.$  
  In principle this reduces the GEP to a standard eigenproblem.  
  However, **explicit inversion of $B$ is numerically unsafe**, especially if $B$ is ill-conditioned. In practice, solvers rely on **QZ/Schur methods** that avoid forming $B^{-1}$ explicitly.

- **Square Pencil Case 2: Symmetric–definite Pencils.**  
  If $A=A^T$ and $B=B^T \succ 0$, the problem admits a **variational characterization** via the generalized Rayleigh quotient.  
  In this setting, eigenvalues are real, eigenvectors can be chosen $B$-orthonormal,  
  and the problem can be reduced to a standard symmetric eigenproblem by **whitening** $B$  
  (via its Cholesky factorization).

# 2. The Generalized Rayleigh Quotient

## 2.1 Definition

Let $A,B \in \mathbb{R}^{n\times n}$ with $B$ symmetric positive definite.  
For any nonzero vector $x \in \mathbb{R}^n$, the **generalized Rayleigh quotient** is

$\qquad R_{A,B}(x) = \dfrac{x^T A x}{x^T B x}.$

**Remark:** The generalized Rayleigh quotient is **homogeneous of degree zero**,  
<div style="padding-left:1cm;">
meaning it is unchanged when $x$ is rescaled.  
Indeed, for any scalar $\alpha \neq 0$,  
$\qquad\displaystyle{ R_{A,B}(\alpha x) 
= \dfrac{(\alpha x)^T A (\alpha x)}{(\alpha x)^T B (\alpha x)}
= \dfrac{\alpha^2 \, x^T A x}{\alpha^2 \, x^T B x}
= R_{A,B}(x).}$<br><br>

Thus $R_{A,B}(x)$ depends only on the **direction** of $x$, not its length.  

Equivalently, we can generate the entire range of $R_{A,B}$  
by restricting to vectors normalized so that $\;x^T B x = 1.$
</div>


## 2.2 Connection to Eigenvalues

- If $(\lambda, x)$ is a generalized eigenpair of $(A,B)$, i.e. $A x = \lambda B x$, then  
  $\qquad R_{A,B}(x) = \lambda.$  

- Conversely, the eigenvalues arise as the **extreme values** of the Rayleigh quotient.  
  Since $R_{A,B}$ is invariant under scaling of $x$, we may restrict our attention to the $B$-unit sphere  
  $\{x \in \mathbb{R}^n : x^T B x = 1\}.$  

  On this constraint set, maximizing or minimizing $R_{A,B}(x) = x^T A x$ subject to $x^T B x = 1$ leads to the Lagrangian  
  $\qquad \mathcal{L}(x,\mu) = x^T A x - \mu (x^T B x - 1).$  

  The first-order condition $\nabla_x \mathcal{L} = 0$ gives  
  $\qquad A x = \mu B x.$  

  Hence, the stationary directions of $R_{A,B}$ are precisely the generalized eigenvectors,  
  and the corresponding stationary values are the generalized eigenvalues.

### 2.2.1 Example ($A$ symmetric, $B$ positive definite)

Let
$\;\;
A = \begin{pmatrix} 2 & 1 \\ 1 & 2 \end{pmatrix}, \quad
B = \begin{pmatrix} 1 & 0 \\ 0 & 2 \end{pmatrix}.
$

For $x = \begin{pmatrix} x_1 \\ x_2 \end{pmatrix}$,

$\qquad 
R_{A,B}(x) = \dfrac{2x_1^2 + 2x_1 x_2 + 2x_2^2}{x_1^2 + 2x_2^2}.
$

- Solving $A x = \lambda B x$ gives  
  $\qquad \lambda_{1,2} = \tfrac{3 \pm \sqrt{3}}{2}.$  

- These are exactly the **stationary values** of $R_{A,B}(x)$ obtained by finding the extrema of $R_{A,B}(x)$   with the contraint $x_1^2 + 2x_2^2=1$, confirming the theory.

Geometrically, $R_{A,B}(x)$ measures the ratio of quadratic forms, and its extrema occur along eigenvectors of $(A,B)$.

In [ ]:
A_ex_1 = np.array([[2.0, 1.0],
                   [1.0, 1.0]])
B_ex_1 = np.array([[1.0, 0.0],
                   [0.0, 2.0]])

viewer_1 = GeneralizedRayleighQuotientViewer(A_ex_1, B_ex_1)
viewer_1.servable()

**Generalized eigenvalue spectrum of the pencil**
$(A,B)$ with
$A=\begin{pmatrix}2&1\\1&2\end{pmatrix}$, 
$B=\begin{pmatrix}1&0\\0&2\end{pmatrix}$.  
$\qquad$ The pencil is regular with $A$ symmetric and $B$ positive definite:  
$\qquad$ The eigenvalues are extrema of the Rayleigh coefficient.

### 2.2.2 Example ($B$ singular)

Let
$\;\;
A = \begin{pmatrix} 2 & 3 \\ 3 & 1 \end{pmatrix}, \quad
B = \begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix}.
$

For $x = \begin{pmatrix} x_1 \\ x_2 \end{pmatrix}$,

$\qquad 
R_{A,B}(x) = \dfrac{2x_1^2 + 6x_1 x_2 + x_2^2}{x_1^2},
\quad \text{(valid when $x_1 \neq 0$).}
$

- Solving $A x = \lambda B x$ gives  
  $\qquad \lambda_1 = -7, \quad \lambda_2 = \infty.$  

- The finite eigenvalue $\lambda=-7$ arises as the stationary value of $R_{A,B}(x)$ when the constraint $x^T B x = x_1^2=1$ is imposed.  
  The second eigenvalue $\lambda=\infty$ reflects the fact that $B$ is singular, and corresponds to directions with $x_1=0$ (for which the denominator $x^T B x$ vanishes and the quotient diverges).

Geometrically, $R_{A,B}(x)$ measures the ratio of quadratic forms, but the singularity of $B$ means the quotient is undefined along some directions.  
The pencil $(A,B)$ illustrates how infinite eigenvalues naturally appear when $B$ is singular.


In [ ]:
A_ex_2 = np.array([[2.0, 3.0],
                   [3.0, 1.0]])
B_ex_2 = np.array([[1.0, 0.0],
                   [0.0, 0.0]])

viewer_2 = GeneralizedRayleighQuotientViewer(A_ex_2, B_ex_2, ylim=(-100,10000))
viewer_2.servable()

**Generalized eigenvalue spectrum of the pencil**
$(A,B)$ with
$A=\begin{pmatrix}2&3\\3&1\end{pmatrix}$, 
$B=\begin{pmatrix}1&0\\0&0\end{pmatrix}$.  
$\qquad$ The pencil is regular but $B$ is singular, so it admits one finite eigenvalue $\lambda=-7$  
$\qquad$ and one eigenvalue at infinity. This illustrates how singular $B$ introduces eigenvalues at $\infty$.

The Rayleigh quotient
$\;\; R_{A,B}(x) = \dfrac{x^T A x}{x^T B x}, \quad x \neq 0,$
is scale-invariant, and its stationary points under $x^T B x = 1$  
correspond to generalized eigenpairs of $(A,B)$.  

But whether these stationary values are **true extremal values** depends strongly on the structure of $A$ and $B$:

1. **General Case (No Symmetry).**  
   - $R_{A,B}(x)$ may take complex values if $A$ or $B$ is not symmetric.
   - Stationary points still satisfy $A x = \lambda B x$, but these are typically **saddle points** of $R_{A,B}$ rather than minima or maxima.  
   - The quotient does not provide a min–max characterization of eigenvalues in this case.

2. **Symmetric–definite Case ($A=A^T$, $B=B^T \succ 0$).**  
   - The quotient is real-valued for all $x \neq 0$.  
   - The generalized eigenvalues are **exactly the extremal values** of $R_{A,B}$:
     $\qquad \lambda_{\min} = \min_{x \neq 0} R_{A,B}(x), \quad
     \lambda_{\max} = \max_{x \neq 0} R_{A,B}(x).$
   - More generally, the **generalized Courant–Fischer theorem** gives
     $\qquad \lambda_k = \min_{\dim \mathcal S = k}\; \max_{x \in \mathcal S\setminus\{0\}} R_{A,B}(x).$
   - Eigenvectors can be chosen $B$-orthonormal, and the extremal property reflects the geometry induced by the $B$-inner product.

3. **$B$ is Indefinite Case.**  
   - The denominator $x^T B x$ can vanish for some $x$, so $R_{A,B}(x)$ can blow up.
   - The variational principle fails; eigenvalues cannot be recovered as extrema of the quotient.

**Summary.**  
- In the **symmetric–definite case** ($A$ symmetric, $B$ symmetric positive definite), eigenvalues are precisely the extremal values of the Rayleigh quotient, with a full min–max characterization.
- In all other cases, eigenvalues are merely stationary values of $R_{A,B}$, without extremal or variational meaning.

## 2.3 Variational Characterization (Symmetric–definite Case)

In general, generalized eigenvalues are stationary values of the Rayleigh quotient, but they need not be extrema.  
A remarkable simplification occurs when $A=A^T$ is symmetric and $B=B^T \succ 0$ is symmetric positive definite.  
In this setting

- all generalized eigenvalues are real and can be ordered
  $\;\; \lambda_1 \le \lambda_2 \le \cdots \le \lambda_n,$  
- the extreme eigenvalues coincide with the global minimum and maximum of $R_{A,B}(x)$ over $x\neq 0$,  
- and more generally each $\lambda_k$ admits a **min–max characterization**:

$\qquad\displaystyle{
\lambda_k \;=\; \min_{\dim \mathcal{S}=k} \;\; \max_{x \in \mathcal{S}\setminus\{0\}} R_{A,B}(x),
}$

where $\mathcal{S}$ ranges over all $k$-dimensional subspaces of $\mathbb{R}^n$.

This is the generalized **Courant–Fischer theorem**, extending the standard variational principle  
to the weighted geometry induced by $B$.


---
**$B$-inner product and orthogonality.**  
Because $B \succ 0$, it defines an inner product
$\;\; \langle x,y\rangle_B = x^T B y.$  
With respect to this geometry,

- eigenvectors of $(A,B)$ can be chosen **$B$-orthonormal**, i.e. $x_i^T B x_j = \delta_{ij}$,  
- the constraint $x^T B x=1$ corresponds to lying on the **$B$-unit sphere**, which is an ellipsoid in the Euclidean picture,  
- and the Rayleigh quotient $R_{A,B}(x)$ measures the ratio of quadratic forms along directions on this ellipsoid.

Geometrically, the variational principle says that the eigenvalues are obtained by optimizing $R_{A,B}(x)$ over these $B$-orthogonal directions — exactly as in the classical symmetric case, but in the geometry defined by $B$.

#### **Example (B-orthonormality).**

Let
$\;\;
A=\begin{pmatrix}2 & 1 \\ 1 & 3\end{pmatrix},\quad
B=\begin{pmatrix}2 & 0 \\ 0 & 1\end{pmatrix}.
$

1. Solve $A x = \lambda B x$.  
   This yields generalized eigenvalues
   $\;\;\lambda_{1,2}=\tfrac{5 \pm \sqrt{5}}{2}$<br>
   and the corresponding (unnormalized) eigenvectors
$\;\;x_{1}=\begin{pmatrix}1 \\ \lambda_1-2\end{pmatrix},\quad
   x_{2}=\begin{pmatrix}1 \\ \lambda_2-2\end{pmatrix}.$

2. Compute their $B$-inner product:
$\;\; (x_{1})^T B\,x_{2} = 0.$  

   Thus the eigenvectors are orthogonal **with respect to $B$**, not in the usual Euclidean sense.

3. After scaling, we can arrange
$\;\; (x_{i})^T B\,x_{j}=\delta_{ij},$  
so the eigenvectors form a **$B$-orthonormal basis** of $\mathbb{R}^2$:<br>
(i.e., compute the Cholesky decomposition $B = L L^T$, and set $s_i = L^T x_i$.)
---

This shows concretely that when $B\succ 0$, the natural geometry for the problem is not Euclidean:  
eigenvectors live on the $B$-unit sphere (an ellipse in this 2x2 example)  
and are orthogonal under the $B$-inner product.

In [ ]:
A_demo = np.array([[2., 1.],
                   [1., 3.]])
B_demo = np.array([[6., 0.],
                   [0., 1.]])

demo_3 = BGeometryViewer(A_demo, B_demo)  # renders two smaller plots side-by-side
demo_3.servable()

# 3. Take Away

- The generalized eigenvalue problem (GEP) $A x = \lambda B x$ is described by the **matrix pencil** $\mathcal{L}(\lambda)=A-\lambda B$.  
- A pencil is **regular** if $\det(A-\lambda B)\not\equiv 0$; otherwise it is singular, and eigenvalues at $\infty$ may occur.  
- The **generalized Rayleigh quotient** $R_{A,B}(x)=\dfrac{x^T A x}{x^T B x}$ is scale-invariant and takes eigenvalues as stationary values.  
- Only in the **symmetric–definite case** ($A=A^T$, $B=B^T \succ 0$) do eigenvalues coincide with **extremal values** of $R_{A,B}$, governed by the generalized Courant–Fischer min–max principle.  
- In this geometry, eigenvectors can be chosen **$B$-orthonormal**, emphasizing that $B$ defines the natural inner product for the problem.  
